# BikewaySim Routing

In [ ]:
import pickle
import pandas as pd
import geopandas as gpd

from bikewaysim.paths import config
from bikewaysim.impedance_calibration import stochastic_optimization, impedance_functions
from bikewaysim.routing import rustworkx_routing_funcs

In [ ]:
links, turns, length_dict, geo_dict, turn_G = rustworkx_routing_funcs.import_calibration_network(config)

# import the improvements
improvements = gpd.read_file(config['bicycle_facilities_fp']/'network_improvements.gpkg',layer='coa')
improvements.rename(columns={'osm_linkid':'linkid'},inplace=True)
improvements[improvements['linkid'].isin(links['linkid'].unique())].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='improvements') # temporary

links = pd.merge(links,improvements.drop(columns=['geometry']),on='linkid',how='left')

# TEMPORARY: this should be solved in the export network step
# set the bike facility attributes to zero if they already exist
bike_facils = ['bike lane','cycletrack','multi use path']
links.loc[links['coa_id'].notna(),bike_facils] = 0

existing_infra = links[(links[bike_facils]==1).any(axis=1)]
existing_infra = existing_infra[['linkid',*bike_facils,'geometry']].drop_duplicates()
existing_infra.to_file(config['bikewaysim_fp']/'map_layers.gpkg',layer='Existing Bicycle Facilities')

In [ ]:
# export a dissolved version of improments for visual clarity
improvements_dissolved = improvements[improvements['linkid'].isin(links['linkid'].unique())].dissolve('improvement').copy()
improvements_dissolved.geometry = improvements_dissolved.buffer(300)
improvements_dissolved.to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='improvements_dissolved') # temporary

# Import ODs

In [ ]:
# import matched ODS (i.e. already have the correct network nodes)
od_matrix = pd.read_csv(config['bikewaysim_fp']/'od_matrix.csv')
ods = list(set(zip(od_matrix['orig_N'],od_matrix['dest_N'])))
starts = [x[0] for x in ods]
ends = [x[1] for x in ods]

## Create Current Travel Time Network

In [ ]:
from importlib import reload
reload(stochastic_optimization)

# run impedance routing on all the provided ODs 170 mins
base_impedance_col = "travel_time_min"
rustworkx_routing_funcs.back_to_base_impedance(base_impedance_col,links,turns,turn_G)
with (config['bikewaysim_fp']/'current_traveltime_novirtual.pkl').open('wb') as fh:
    pickle.dump(turn_G,fh)
added_nodes = rustworkx_routing_funcs.add_virtual_edges(starts,ends,links,turns,turn_G)
with (config['bikewaysim_fp']/'current_traveltime.pkl').open('wb') as fh:
    pickle.dump(turn_G,fh)
rustworkx_routing_funcs.remove_virtual_links(added_nodes,turn_G)

In [ ]:
# traveltime_links = links.copy()
# #export the current network
# traveltime_links[['linkid','reverse_link','travel_time_min','multiplier','link_cost','geometry']].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='traveltime_network')

# Import Calibration Result to Get Coefficients

In [ ]:
with (config['calibration_fp'] / 'results/bootstrap_final,validation,0.pkl').open('rb') as fh:
    bootstrap_model = pickle.load(fh)
beta_cols = [x['col'] for x in bootstrap_model['betas_tup'] if x['type'] == 'link']
betas_tup = bootstrap_model['betas_tup']
betas = [x['beta'] for x in betas_tup] # get betas

# Least Impedance Routing


In [ ]:
# run impedance routing on all the provided ODs
base_impedance_col = "travel_time_min"
rustworkx_routing_funcs.back_to_base_impedance(base_impedance_col,links,turns,turn_G)
#update impedances
print(betas)
_ = rustworkx_routing_funcs.impedance_update(
    betas,betas_tup,
    impedance_functions.link_impedance_function,
    base_impedance_col,
    None,
    impedance_functions.turn_impedance_function,
    links,turns,turn_G)
with (config['bikewaysim_fp']/'current_impedance_novirtual.pkl').open('wb') as fh:
    pickle.dump(turn_G,fh)
added_nodes = rustworkx_routing_funcs.add_virtual_edges(starts,ends,links,turns,turn_G)
with (config['bikewaysim_fp']/'current_impedance.pkl').open('wb') as fh:
    pickle.dump(turn_G,fh)
rustworkx_routing_funcs.remove_virtual_links(added_nodes,turn_G)

In [ ]:
current_links = links.copy()
#export the current network
current_links[['linkid','reverse_link']+beta_cols+['travel_time_min','multiplier','link_cost','geometry']].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='current_network')

# Repeat Least Impedance Routing with Network Improvements


In [ ]:
print(links['bike lane'].sum())
print(links['cycletrack'].sum())

In [ ]:
# create the neccessary attributes for impednaces
links.loc[links['improvement']=='bike lane','bike lane'] = 1
links.loc[links['improvement']=='cycletrack','cycletrack'] = 1
# links['multi use path'] = (links['improvement'] == 'multi use path').astype(int) # this one would require modifying the other attributes

In [ ]:
print(links['bike lane'].sum())
print(links['cycletrack'].sum())

In [ ]:
from importlib import reload
reload(stochastic_optimization)
 
# run impedance routing on all the provided ODs
base_impedance_col = "travel_time_min"
rustworkx_routing_funcs.back_to_base_impedance(base_impedance_col,links,turns,turn_G)

betas_tup = bootstrap_model['betas_tup']
betas = [x['beta'] for x in betas_tup] # get betas

#update impedances
print(betas)
_ = rustworkx_routing_funcs.impedance_update(
    betas,betas_tup,
    impedance_functions.link_impedance_function,
    base_impedance_col,
    None,
    impedance_functions.turn_impedance_function,
    links,turns,turn_G)
with (config['bikewaysim_fp']/'future_impedance_novirtual.pkl').open('wb') as fh:
    pickle.dump(turn_G,fh)
added_nodes = rustworkx_routing_funcs.add_virtual_edges(starts,ends,links,turns,turn_G)
with (config['bikewaysim_fp']/'future_impedance.pkl').open('wb') as fh:
    pickle.dump(turn_G,fh)
rustworkx_routing_funcs.remove_virtual_links(added_nodes,turn_G)

In [ ]:
future_links = links.copy()
#export the current network
future_links[['linkid','reverse_link']+beta_cols+['travel_time_min','multiplier','link_cost','geometry']].to_file(config['bikewaysim_fp']/'framework_results.gpkg',layer='future_network')